## Validate PR #376 to adjust acqs "maxmag" to limit search hits to < 50

This notebook reselects the star catalogs from JUL0422B, JUL1122A, JUL1822A, and
JUL2522A at a CCD temperature of -4.0 C using both the current proseco master 
(eeb6b72) and [PR #376](https://github.com/sot/proseco/pull/376) (baaa6de).

With these the following are shown:
- Diff of catalogs that are different
- Diff of sparkles critical warnings

### Summary
The catalog changes are mostly minor:
- Small reductions in MAXMAG.
- Some box sizes are reduced.
- A few instances where box sizes increase, likely due to different starting
  conditions leading to a different local minimum.
- One instance of a different acq star being selected.

The critical messages diffs are also minor. No new criticals, with only small
changes in the acquisition P2.

In [1]:
from tqdm import tqdm
import numpy as np
import gzip
import os
import mock
import pickle
import sys
from pathlib import Path
import difflib

from kadi.commands.commands_v2 import APPROVED_LOADS_OCCWEB_DIR
from kadi.occweb import get_occweb_page

sys.path.insert(0, os.path.expanduser("~/git/proseco"))
from proseco import get_aca_catalog
from proseco.diff import catalog_diff
from sparkles import run_aca_review

In [2]:
def get_pickle_path(load_name: str) -> Path:
    year = '20' + load_name[5:7]
    month_name = load_name[:3]
    pkl_path = (
        APPROVED_LOADS_OCCWEB_DIR 
        / year 
        / month_name 
        / load_name 
        / 'output'
        / f'{load_name}_proseco.pkl.gz'
        )
    return pkl_path

In [3]:
get_pickle_path('JUN2720A')

PosixPath('FOT/mission_planning/PRODUCTS/APPR_LOADS/2020/JUN/JUN2720A/output/JUN2720A_proseco.pkl.gz')

In [4]:
load_names = [
  'JUL0422B', 'JUL1122A', 'JUL1822A', 'JUL2522A'
]

In [5]:
load_name = load_names[0]
path = get_pickle_path(load_name)
print(path)
pkl_bytes = get_occweb_page(path, binary=True, cache=True)
acas = pickle.loads(gzip.decompress(pkl_bytes))

FOT/mission_planning/PRODUCTS/APPR_LOADS/2022/JUL/JUL0422B/output/JUL0422B_proseco.pkl.gz


In [6]:
acas.keys()

dict_keys([25315.0, 45400.0, 23779.0, 23904.0, 23905.0, 23906.0, 24414.0, 25532.0, 45399.0, 45398.0, 45397.0, 45396.0, 45395.0, 45394.0, 26440.0, 25709.0, 25306.0, 24369.0, 23837.0, 26448.0, 23908.0, 24450.0, 45393.0, 45392.0, 25623.0, 26447.0, 26449.0, 25721.0, 26450.0, 24325.0, 23907.0, 23724.0, 24410.0, 45391.0])

In [7]:
acas[25315].call_args

{'obsid': 25315.0,
 'att': <Quat q1=0.08554695 q2=-0.82394821 q3=0.54016793 q4=0.14835660>,
 'date': '2022:185:01:33:03.737',
 'detector': 'ACIS-S',
 'dither_acq': array([7.9992, 7.9992]),
 'dither_guide': array([7.9992, 7.9992]),
 'man_angle': 33.62856232032029,
 'n_acq': 8,
 'n_fid': 3,
 'n_guide': 5,
 'sim_offset': 0.0,
 'focus_offset': 0.0,
 't_ccd_acq': -11.49184658381997,
 't_ccd_guide': -11.328734753812505,
 't_ccd_penalty_limit': -6.2}

In [8]:
call_args = acas[25315].call_args.copy()
call_args['t_ccd_acq'] = -4.0
call_args['t_ccd_guide'] = -4.0
call_args['t_ccd_penalty_limit'] = 0.0
aca_new = get_aca_catalog(**call_args)

In [9]:
aca_new

slot,idx,id,type,sz,p_acq,mag,maxmag,yang,zang,dim,res,halfw
int64,int64,int64,str3,str3,float64,float64,float64,float64,float64,int64,int64,int64
0,1,2,FID,8x8,0.000,7.00,8.00,-773.20,-1742.03,1,1,25
1,2,4,FID,8x8,0.000,7.00,8.00,2140.23,166.63,1,1,25
2,3,5,FID,8x8,0.000,7.00,8.00,-1826.28,160.17,1,1,25
3,4,189143880,BOT,6x6,0.976,8.01,9.51,-1708.91,1263.11,12,1,80
4,5,189140528,BOT,6x6,0.974,8.25,9.75,1176.06,-213.18,12,1,80
5,6,189147808,BOT,6x6,0.970,8.51,10.01,-2212.80,-2165.66,12,1,80
6,7,189140696,GUI,6x6,0.000,8.45,9.95,1104.30,-197.69,1,1,25
0,8,189151784,ACQ,6x6,0.121,10.40,11.09,1856.98,-131.73,12,1,80
1,9,189137760,ACQ,6x6,0.117,10.51,11.20,2327.71,1524.09,8,1,60


In [10]:
with mock.patch.dict(os.environ, {"PROSECO_IGNORE_MAXMAGS_CONSTRAINTS": "1"}):
    aca_legacy = get_aca_catalog(**call_args)

In [11]:
aca_legacy

slot,idx,id,type,sz,p_acq,mag,maxmag,yang,zang,dim,res,halfw
int64,int64,int64,str3,str3,float64,float64,float64,float64,float64,int64,int64,int64
0,1,2,FID,8x8,0.000,7.00,8.00,-773.20,-1742.03,1,1,25
1,2,4,FID,8x8,0.000,7.00,8.00,2140.23,166.63,1,1,25
2,3,5,FID,8x8,0.000,7.00,8.00,-1826.28,160.17,1,1,25
3,4,189143880,BOT,6x6,0.976,8.01,9.51,-1708.91,1263.11,12,1,80
4,5,189140528,BOT,6x6,0.974,8.25,9.75,1176.06,-213.18,12,1,80
5,6,189147808,BOT,6x6,0.970,8.51,10.01,-2212.80,-2165.66,12,1,80
6,7,189140696,GUI,6x6,0.000,8.45,9.95,1104.30,-197.69,1,1,25
0,8,189151784,ACQ,6x6,0.121,10.40,11.20,1856.98,-131.73,12,1,80
1,9,189137760,ACQ,6x6,0.058,10.51,11.20,2327.71,1524.09,16,1,100


In [12]:
aca_new.acqs

id,ra,dec,yang,zang,row,col,mag,mag_err,color,idx,type,halfw,p_acq,slot,maxmag,dim,res,sz
int32,float64,float64,float64,float64,float64,float64,float32,float64,float32,int64,str3,int64,float64,int64,float64,int64,int64,str3
189143880,179.367982,19.98424,-1708.91,1263.11,350.20,258.31,8.01,0.04,0.83,0,BOT,80,0.976,3,9.51,12,1,6x6
189140528,178.630635,19.41113,1176.06,-213.18,-229.87,-37.43,8.25,0.05,0.65,1,BOT,80,0.974,4,9.75,12,1,6x6
189147808,178.495044,20.49001,-2212.80,-2165.66,452.39,-433.49,8.51,0.05,1.49,3,BOT,80,0.970,5,10.01,12,1,6x6
189151784,178.573629,19.22839,1856.98,-131.73,-367.19,-20.94,10.40,0.21,1.01,5,ACQ,80,0.121,0,11.09,12,1,6x6
189137760,178.965973,18.92651,2327.71,1524.09,-463.17,313.21,10.51,0.21,0.85,6,ACQ,60,0.117,1,11.20,8,1,6x6
189151952,178.486281,19.10509,2381.99,-229.86,-473.85,-40.60,10.54,0.22,1.07,7,ACQ,60,0.112,2,11.20,8,1,6x6
189152000,178.364996,19.55995,1038.61,-1252.85,-202.94,-246.26,10.63,0.21,1.38,8,ACQ,60,0.098,6,11.20,8,1,6x6
189138488,178.518973,19.69366,390.63,-962.25,-72.59,-187.81,10.88,0.26,0.99,9,ACQ,60,0.058,7,11.20,8,1,6x6


In [13]:
catalog_diff(aca_legacy, aca_new, names='idx id slot type mag maxmag halfw')

f,1,idx id slot type mag maxmag halfw,f,1,idx id slot type mag maxmag halfw
,2,--- --------- ---- ---- ----- ------ -----,,2,--- --------- ---- ---- ----- ------ -----
,3,1 2 0 FID 7.00 8.00 25,,3,1 2 0 FID 7.00 8.00 25
,4,2 4 1 FID 7.00 8.00 25,,4,2 4 1 FID 7.00 8.00 25
,5,3 5 2 FID 7.00 8.00 25,,5,3 5 2 FID 7.00 8.00 25
,6,--- --------- ---- ---- ----- ------ -----,,6,--- --------- ---- ---- ----- ------ -----
,7,5 189140528 4 GU* 8.25 9.75 25,,7,5 189140528 4 GU* 8.25 9.75 25
,8,7 189140696 6 GUI 8.45 9.95 25,,8,7 189140696 6 GUI 8.45 9.95 25
,9,4 189143880 3 GU* 8.01 9.51 25,,9,4 189143880 3 GU* 8.01 9.51 25
,10,6 189147808 5 GU* 8.51 10.01 25,,10,6 189147808 5 GU* 8.51 10.01 25
,11,--- --------- ---- ---- ----- ------ -----,,11,--- --------- ---- ---- ----- ------ -----


In [14]:
aca_new.acqs.get_log_p_2_or_fewer()

-1.3369019423935624

In [15]:
aca_legacy.acqs.get_log_p_2_or_fewer()

-1.2696535720978193

In [16]:
def reselect_load(load_name, t_ccd=None):
    path = get_pickle_path(load_name)
    print(f"Getting {load_name} from {path}")
    pkl_bytes = get_occweb_page(path, binary=True, cache=True)
    acas = pickle.loads(gzip.decompress(pkl_bytes))    

    acas_new = []
    acas_legacy = []

    def is_equal(t1, t2):
        if len(t1) != len(t2):
            return False
        return all(np.all(c) for c in t1.values_equal(t2).itercols())

    for aca in tqdm(acas.values()):
        call_args = aca.call_args.copy()
        if t_ccd is not None:
            call_args['t_ccd_acq'] = t_ccd
            call_args['t_ccd_guide'] = t_ccd
        call_args['t_ccd_penalty_limit'] = 10.0
        aca_new = get_aca_catalog(**call_args)
        with mock.patch.dict(os.environ, {"PROSECO_IGNORE_MAXMAGS_CONSTRAINTS": "1"}):
            aca_legacy = get_aca_catalog(**call_args)
        if not is_equal(aca_new, aca_legacy):
            acas_new.append(aca_new)
            acas_legacy.append(aca_legacy)

    return acas_new, acas_legacy

In [17]:
def get_criticals(acas):
    text_criticals = {}
    acars = [aca.get_review_table() for aca in acas]
    for acar in acars:
        lines = []
        acar.run_aca_review()
        criticals = acar.messages >= "critical"
        if criticals:
            for msg in criticals:
                lines.append(msg['category'].capitalize() + ': ' + msg['text'])
            text_criticals[acar.obsid] = lines
    return text_criticals


In [18]:
def get_diff_catalog(acas_new, acas_legacy):    
    diff = catalog_diff(acas_legacy, acas_new, 
                        names='idx id slot type mag maxmag halfw',
                        labels=[f"Obsid {aca.obsid}" for aca in acas_new])
    return diff

In [19]:
def process_load(load_name, t_ccd=None):
    acas_new, acas_legacy = reselect_load(load_name, t_ccd)
    diff_catalog = get_diff_catalog(acas_new, acas_legacy)
    criticals_new = get_criticals(acas_new)
    criticals_legacy = get_criticals(acas_legacy)
    obsids = set(criticals_new) | set(criticals_legacy)
    for obsid in obsids:
        if obsid not in criticals_new:
            criticals_new[obsid] = []
        if obsid not in criticals_legacy:
            criticals_legacy[obsid] = []    
    diff_criticals = get_diff_criticals(criticals_new, criticals_legacy)
    return diff_catalog, diff_criticals

In [20]:
from proseco.diff import CatalogDiff, HTML_FOOTER, HTML_HEADER

HTML_HEADER = """
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
          "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
<head>
    <meta http-equiv="Content-Type"
          content="text/html; charset=utf-8" />
    <title></title>
    <style type="text/css">
        table.diff {font-family:Courier; border:medium;}
        table.diff td {padding-top: 0em; padding-bottom: 0em;
                       padding-right: 0.5em; padding-left: 0.5em}
        .diff_header {background-color:#e0e0e0}
        td.diff_header {text-align:right}
        .diff_next {background-color:#c0c0c0}
        .diff_add {background-color:#aaffaa}
        .diff_chg {background-color:#ffff77}
        .diff_sub {background-color:#ffaaaa}
        td { text-align: left; }
    </style>
</head>
<body>
"""

def get_lines(criticals):
    lines = []
    for obsid, crits in criticals.items():
        lines.append("-------------------------------")
        lines.append(f"{obsid}")
        lines.append("-------------------------------")
        lines.extend(crits)
    return lines

def get_diff_criticals(criticals_new, criticals_legacy):
    lines_new = get_lines(criticals_new)
    lines_legacy = get_lines(criticals_legacy)

    differ = difflib.HtmlDiff()
    text_diff = differ.make_table(lines_legacy, lines_new)

    return CatalogDiff(HTML_HEADER + text_diff + HTML_FOOTER, is_html=True)

## JUL0422B

In [21]:
print(load_names[0])

JUL0422B


In [22]:
diff_catalog, diff_criticals = process_load(load_names[0], t_ccd=-4.0)

Getting JUL0422B from FOT/mission_planning/PRODUCTS/APPR_LOADS/2022/JUL/JUL0422B/output/JUL0422B_proseco.pkl.gz


100%|██████████| 34/34 [00:25<00:00,  1.34it/s]


In [23]:
diff_catalog

f,1,idx id slot type mag maxmag halfw,f,1,idx id slot type mag maxmag halfw
,2,--- --------- ---- ---- ----- ------ -----,,2,--- --------- ---- ---- ----- ------ -----
,3,1 2 0 FID 7.00 8.00 25,,3,1 2 0 FID 7.00 8.00 25
,4,2 4 1 FID 7.00 8.00 25,,4,2 4 1 FID 7.00 8.00 25
,5,3 5 2 FID 7.00 8.00 25,,5,3 5 2 FID 7.00 8.00 25
,6,--- --------- ---- ---- ----- ------ -----,,6,--- --------- ---- ---- ----- ------ -----
,7,5 189140528 4 GU* 8.25 9.75 25,,7,5 189140528 4 GU* 8.25 9.75 25
,8,7 189140696 6 GUI 8.45 9.95 25,,8,7 189140696 6 GUI 8.45 9.95 25
,9,4 189143880 3 GU* 8.01 9.51 25,,9,4 189143880 3 GU* 8.01 9.51 25
,10,6 189147808 5 GU* 8.51 10.01 25,,10,6 189147808 5 GU* 8.51 10.01 25
,11,--- --------- ---- ---- ----- ------ -----,,11,--- --------- ---- ---- ----- ------ -----


In [24]:
diff_criticals

f,1,-------------------------------,f,1,-------------------------------
,2,25315.0,,2,25315.0
,3,-------------------------------,,3,-------------------------------
,4,"Critical: Guide indexes [5, 7] clustered within 500"" radius",,4,"Critical: Guide indexes [5, 7] clustered within 500"" radius"
n,5,Critical: P2: 1.31 less than 2.0 for OR,n,5,Critical: P2: 1.37 less than 2.0 for OR
,6,-------------------------------,,6,-------------------------------
,7,45395.0,,7,45395.0
,8,-------------------------------,,8,-------------------------------
,9,Critical: P2: 2.74 less than 3.0 for ER,,9,Critical: P2: 2.74 less than 3.0 for ER
,10,Critical: ER count of guide stars 5.37 < 6.0,,10,Critical: ER count of guide stars 5.37 < 6.0
,11,-------------------------------,,11,-------------------------------


## JUL1122A

In [25]:
print(load_names[1])

JUL1122A


In [26]:
diff_catalog, diff_criticals = process_load(load_names[1], t_ccd=-4.0)

Getting JUL1122A from FOT/mission_planning/PRODUCTS/APPR_LOADS/2022/JUL/JUL1122A/output/JUL1122A_proseco.pkl.gz


100%|██████████| 37/37 [00:28<00:00,  1.30it/s]


In [27]:
diff_catalog

f,1,idx id slot type mag maxmag halfw,f,1,idx id slot type mag maxmag halfw
,2,--- ---------- ---- ---- ----- ------ -----,,2,--- ---------- ---- ---- ----- ------ -----
,3,3 1112147112 2 GU* 8.26 9.76 25,,3,3 1112147112 2 GU* 8.26 9.76 25
,4,7 1112147456 6 GU* 9.42 10.92 25,,4,7 1112147456 6 GU* 9.42 10.92 25
,5,8 1112148208 7 GUI 9.25 10.75 25,,5,8 1112148208 7 GUI 9.25 10.75 25
,6,5 1112148368 4 GU* 8.50 10.00 25,,6,5 1112148368 4 GU* 8.50 10.00 25
,7,6 1112149896 5 GU* 8.77 10.27 25,,7,6 1112149896 5 GU* 8.77 10.27 25
,8,4 1112152136 3 GU* 8.27 9.77 25,,8,4 1112152136 3 GU* 8.27 9.77 25
,9,2 1112152776 1 GU* 8.13 9.63 25,,9,2 1112152776 1 GU* 8.13 9.63 25
,10,1 1112156360 0 GU* 8.10 9.60 25,,10,1 1112156360 0 GU* 8.10 9.60 25
,11,--- ---------- ---- ---- ----- ------ -----,,11,--- ---------- ---- ---- ----- ------ -----


In [28]:
diff_criticals

f,1,-------------------------------,f,1,-------------------------------
,2,25105.0,,2,25105.0
,3,-------------------------------,,3,-------------------------------
,4,Critical: P2: 1.54 less than 2.0 for OR,,4,Critical: P2: 1.54 less than 2.0 for OR
,5,Critical: OR count of guide stars 3.25 < 4.0,,5,Critical: OR count of guide stars 3.25 < 4.0
,6,-------------------------------,,6,-------------------------------
,7,25968.0,,7,25968.0
,8,-------------------------------,,8,-------------------------------
,9,Critical: OR count of guide stars 3.79 < 4.0,,9,Critical: OR count of guide stars 3.79 < 4.0
,10,-------------------------------,,10,-------------------------------
,11,25170.0,,11,25170.0


## JUL1822A

In [29]:
print(load_names[2])

JUL1822A


In [30]:
diff_catalog, diff_criticals = process_load(load_names[2], t_ccd=-4.0)

Getting JUL1822A from FOT/mission_planning/PRODUCTS/APPR_LOADS/2022/JUL/JUL1822A/output/JUL1822A_proseco.pkl.gz


100%|██████████| 48/48 [00:35<00:00,  1.36it/s]


In [31]:
diff_catalog

f,1,idx id slot type mag maxmag halfw,f,1,idx id slot type mag maxmag halfw
,2,--- --------- ---- ---- ----- ------ -----,,2,--- --------- ---- ---- ----- ------ -----
n,3,1 2 0 FID 7.00 8.00 25,n,3,1 3 0 FID 7.00 8.00 25
,4,2 4 1 FID 7.00 8.00 25,,4,2 4 1 FID 7.00 8.00 25
,5,3 5 2 FID 7.00 8.00 25,,5,3 5 2 FID 7.00 8.00 25
,6,--- --------- ---- ---- ----- ------ -----,,6,--- --------- ---- ---- ----- ------ -----
,7,6 122160176 5 GU* 9.33 10.83 25,,7,6 122160176 5 GU* 9.33 10.83 25
,8,7 122162400 6 GU* 9.35 10.85 25,,8,7 122162400 6 GU* 9.35 10.85 25
,9,8 122162760 7 GU* 9.57 11.07 25,,9,8 122162760 7 GU* 9.57 11.07 25
,10,4 122166800 3 GU* 7.28 8.78 25,,10,4 122166800 3 GU* 7.28 8.78 25
,11,5 122689072 4 GU* 7.64 9.14 25,,11,5 122689072 4 GU* 7.64 9.14 25


In [32]:
diff_criticals

t,1,-------------------------------,t,1,-------------------------------
,2,26458.0,,2,26458.0
,3,-------------------------------,,3,-------------------------------
,4,Critical: P2: 1.82 less than 2.0 for OR,,4,Critical: P2: 1.82 less than 2.0 for OR
,5,-------------------------------,,5,-------------------------------
,6,45372.0,,6,45372.0
,7,-------------------------------,,7,-------------------------------
,8,Critical: ER count of 9th (8.3 for -4.0C) mag guide stars 2.60 < 3.0,,8,Critical: ER count of 9th (8.3 for -4.0C) mag guide stars 2.60 < 3.0
,9,Critical: ER count of guide stars 5.28 < 6.0,,9,Critical: ER count of guide stars 5.28 < 6.0
,10,-------------------------------,,10,-------------------------------
,11,45371.0,,11,45371.0


## JUL2522A

In [33]:
print(load_names[3])


JUL2522A


In [34]:
diff_catalog, diff_criticals = process_load(load_names[3], t_ccd=-4.0)


Getting JUL2522A from FOT/mission_planning/PRODUCTS/APPR_LOADS/2022/JUL/JUL2522A/output/JUL2522A_proseco.pkl.gz


100%|██████████| 33/33 [00:24<00:00,  1.36it/s]


In [35]:
diff_catalog


f,1,idx id slot type mag maxmag halfw,f,1,idx id slot type mag maxmag halfw
,2,--- --------- ---- ---- ----- ------ -----,,2,--- --------- ---- ---- ----- ------ -----
,3,1 2 0 FID 7.00 8.00 25,,3,1 2 0 FID 7.00 8.00 25
,4,2 4 1 FID 7.00 8.00 25,,4,2 4 1 FID 7.00 8.00 25
,5,3 5 2 FID 7.00 8.00 25,,5,3 5 2 FID 7.00 8.00 25
,6,--- --------- ---- ---- ----- ------ -----,,6,--- --------- ---- ---- ----- ------ -----
,7,5 890504456 4 GU* 8.17 9.67 25,,7,5 890504456 4 GU* 8.17 9.67 25
,8,6 890506624 5 GU* 8.95 10.45 25,,8,6 890506624 5 GU* 8.95 10.45 25
,9,4 890507552 3 GU* 7.39 8.89 25,,9,4 890507552 3 GU* 7.39 8.89 25
,10,--- --------- ---- ---- ----- ------ -----,,10,--- --------- ---- ---- ----- ------ -----
,11,8 890504416 1 ACQ 9.52 11.02 60,,11,8 890504416 1 ACQ 9.52 11.02 60


In [36]:
diff_criticals

f,1,-------------------------------,f,1,-------------------------------
,2,24762.0,,2,24762.0
,3,-------------------------------,,3,-------------------------------
,4,"Critical: Guide indexes [4] clustered within 500"" radius",,4,"Critical: Guide indexes [4] clustered within 500"" radius"
,5,"Critical: Guide indexes [5] clustered within 500"" radius",,5,"Critical: Guide indexes [5] clustered within 500"" radius"
,6,"Critical: Guide indexes [6] clustered within 500"" radius",,6,"Critical: Guide indexes [6] clustered within 500"" radius"
,7,Critical: OR count of guide stars 3.00 < 4.0,,7,Critical: OR count of guide stars 3.00 < 4.0
,8,-------------------------------,,8,-------------------------------
,9,23847.0,,9,23847.0
,10,-------------------------------,,10,-------------------------------
n,11,Critical: P2: 1.51 less than 2.0 for OR,n,11,Critical: P2: 1.52 less than 2.0 for OR
